In [1]:
import pandas as pd
import numpy as np


In [2]:
# load the df
df=pd.read_csv('Consumer_Complaints_usb.csv')

### import text processor

In [ ]:
import TextPreprocessing as textpro
text_processor=textpro.TextNormalization()
df=text_processor.format_headers(df)
df.columns


In [ ]:
df=text_processor.header_to_title(df)
df.columns


In [ ]:
df=text_processor.normalize_dataframe_text(df,normalize_unicode=True)

### import date module

In [ ]:
import DateTime as datenorm
date_non_conform_detector=datenorm.DateTime()
date_columns=['Date Received','Date Sent To Company']

date_non_conform_detector.detect_invalid_dates(df,date_columns,yearfirst=True)
print('invalid dates by column')
for k,v in date_non_conform_detector.invalid_date_map.items():
    print(f"{k}: {v}")

In [ ]:
df=date_non_conform_detector.normalize_datetime(df,date_columns,yearfirst=True)

### import the semantic-sentence  class

In [3]:
import SemanticSentenceMatching as ssm
sentence_matcher=ssm.SemanticSentenceMatching()

create a similarity score df for the 'Tags' column [as an example] and put it in the dir

In [4]:
sentence_matcher.create_similarity_cvs_dir(df, 'Tags', 'similarity_dfs',filter=0.7)

annotate in streamlit

In [ ]:
# call sentence_matcher.run_streamlit_annotator() or bash streamlit run TrueFalse_Dataframe_Annotator.py
# if call is made, the .ipynb kernel often needs to be reset to proceed with .ipynb use

or annotate in .ipynb using get_similarity_comparison_df() & get_user_annotation()

In [10]:
tags_df=sentence_matcher.get_similarity_comparison_df(df['Tags'],filter=.0)
bool_tags=sentence_matcher.get_user_annotation(tags_df)
print("The similarity df filtere by 'is_similar'==True")
display(bool_tags)
bool_tags.to_csv('similarity_dfs\Tags_similarity_annotation_df.csv',index=False)

,is_similar,column_a,column_b,similarity_score
0,True,Older American,"Older American, Servicemember",0.883682
1,False,Servicemember,"Older American, Servicemember",0.788231
2,False,Servicemember,Older American,0.604816


The similarity df filtere by 'is_similar'==True


,column_a,column_b,similarity_score,is_similar


zip column headers to dirs, group all similar entries, and update dataframe  

##  WARNING: zip_paths_and_files_by_extension() does not discriminate. Every file in the directory will be used to update the man dataframe

In [ ]:
#zip column headers to dirs
zipped_features_to_filepaths = sentence_matcher.zip_paths_and_files_by_extension(directory='./similarity_dfs', extension='csv')


#group all similar entries
similar_entries_maps = sentence_matcher.get_similar_entries(zipped_features_to_filepaths)


In [13]:

# a look at before 
print('before'.upper())
for col in similar_entries_maps:
    print(col)
    print(df[col].nunique())

#update dataframe 
new_df=sentence_matcher.update_feature_columns_based_on_similar_entries(df, similar_entries_maps)

# a look at after
print('after'.upper())
for col in similar_entries_maps:
    print(col)
    print(newdf[col].nunique())
del new_df

BEFORE
AFTER
